In [2]:
import pandas as pd
from flask import Flask, request, jsonify, render_template

d = pd.read_csv('shoe_size.csv')

In [3]:
#change the shoe-size column to the integer type
d['shoe_size'] = d['shoe_size'].astype(int)
d.tail(10)

#convert 'M' to 1, and 'F' to 2 for the sex type to train the Ml module
d['sex_no'] = 0
d.loc[d['sex'] == 'M','sex_no'] = 1
d.loc[d['sex'] == 'F', 'sex_no'] = 2

d['sex_no'] = d['sex_no'].astype(int)

d.head(5)

,height,sex,shoe_size,sex_no
0,160,F,40,2
1,171,F,39,2
2,174,F,39,2
3,176,F,40,2
4,195,M,46,1


In [4]:
#Now lets build a model

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

lr = LinearRegression()

In [5]:
X = d[['height','sex_no']]
y = d['shoe_size']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
#lets fit the model 

lr.fit(X_train,y_train)

/Users/admin/anaconda3/envs/dsr/lib/python3.6/site-packages/sklearn/linear_model/base.py:509: RuntimeWarning: internal gelsd driver lwork query error, required iwork dimension not returned. This is likely the result of LAPACK bug 0038, fixed in LAPACK 3.2.2 (released July 21, 2010). Falling back to 'gelss' driver.
  linalg.lstsq(X, y)


LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [7]:
preds = lr.predict(X_test)

In [8]:
from sklearn.metrics import mean_squared_error

mean_squared_error(y_test,preds)

9.042582839534953

In [9]:
#lets build random forest

from sklearn.ensemble import RandomForestRegressor

/Users/admin/anaconda3/envs/dsr/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [10]:
rf = RandomForestRegressor(max_depth=4, random_state=0,n_estimators=100)

rf.fit(X_train,y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=4,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [11]:
pred = rf.predict(X_test)

mean_squared_error(y_test,pred)

2.0936744373672047

In [12]:
#fit the model on all the data

rf.fit(X,y)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=4,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=0, verbose=0, warm_start=False)

In [13]:
# save the model to disk
import pickle

filename = 'model.pkl'
pickle.dump(rf, open(filename, 'wb'))

In [15]:
# some time later...
 
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)

0.8708781196801274
